# Introduction to Ethereum for Developers

## Author:  Braun E. Brelin
(C) 2021


This notebook will present the *Ethereum* Blockchain environment.  Ethereum is the first *global* application deployed.  The desisgners of Ethereum call it a "World computer".  We will explore this application and learn how it works internally as well as how to develop distributed applications (known as DApps) for this system. 

## What is a blockchain?

A good definition of a blockchain is a “Cryptographically secure transactional singleton machine with shared-state.”
What does this mean?

- “Cryptographically secure” means that the creation of digital currency is secured by complex mathematical algorithms that are very hard to break. These algorithms make it nearly impossible to cheat the system (e.g. create fake transactions, erase transactions, etc.)
- “Transactional singleton machine” means that there’s a single canonical instance of the machine responsible for all the transactions being created in the system. In other words, there’s a single global truth that everyone believes in.
- “With shared-state” means that the state stored on this machine is shared and open to everyone.

Ethereum is an exxample of a system that implements a blockchain. 

## The Ethereum blockchain paradigm explained
The Ethereum blockchain is essentially a transaction-based state machine. In computer science, a state machine refers to something that will read a series of inputs and, based on those inputs, will transition to a new state.

The following figure gives a graphical example of a simple state machine. 

### Figure 1.
<img src="./graphics/State_Machine_Example.png">

The Ethereum initial state is called the *genesis state*.  This was the initial state before any transactions were computed and added. When transactions are executed, this genesis state transitions into some final state. At any point in time, this final state represents the current state of Ethereum.
The following figure is a graphical representation of this idea. 

### Figure 2.

<img src = "./graphics/genesis_transition_example.png">


The state of Ethereum has millions of transactions. These transactions are grouped into “blocks.” A block contains a series of transactions, and each block is chained together with its previous block. We can represent this graphically.

### Figure 3. 
<img src="./graphics/Blockchain_example1.png"/> 

To cause a transition from one state to the next, a transaction must be valid. For a transaction to be considered valid, it must go through a validation process known as mining. Mining is when a group of nodes (i.e. computers) expend their compute resources to create a block of valid transactions.

Any node on the network that declares itself as a miner can attempt to create and validate a block. Lots of miners from around the world try to create and validate blocks at the same time. Each miner provides a mathematical “proof” when submitting a block to the blockchain, and this proof acts as a guarantee: if the proof exists, the block must be valid.

For a block to be added to the main blockchain, the miner must prove it faster than any other competitor miner. The process of validating each block by having a miner provide a mathematical proof is known as a “proof of work.”

A miner who validates a new block is rewarded with a certain amount of value for doing this work. What is that value? The Ethereum blockchain uses an intrinsic digital token called “Ether.” Every time a miner proves a block, new Ether tokens are generated and awarded.

What guarantees that everyone sticks to one chain of blocks? How can we be sure that there doesn’t exist a subset of miners who will decide to create their own chain of blocks?

Earlier, we defined a blockchain as a **transactional singleton machine with shared-state.** Using this definition, we can understand the correct current state is a single global truth, which everyone must accept. Having multiple states (or chains) would ruin the whole system, because it would be impossible to agree on which state was the correct one. If the chains were to diverge, you might own 10 coins on one chain, 20 on another, and 40 on another. In this scenario, there would be no way to determine which chain was the most “valid.”

Whenever multiple paths are generated, a “fork” occurs. We typically want to avoid forks, because they disrupt the system and force people to choose which chain they “believe” in.

A blockchain fork may look like this.

### Figure 4. 

<img src="./graphics/fork_diagram1.png"/>


### The GHOST protocol

To determine which path is most valid and prevent multiple chains, Ethereum uses a mechanism called the “GHOST protocol.”
GHOST is an acronym for “**G**reedy **H**eaviest **O**bserved **S**ub**t**ree”

The GHOST protocol says we must pick the path that has had the most computation done upon it. One way to determine that path is to use the block number of the most recent block (the “leaf block”), which represents the total number of blocks in the current path (not counting the genesis block). The higher the block number, the longer the path and the greater the mining effort that must have gone into arriving at the leaf. Using this reasoning allows us to agree on the canonical version of the current state.

### Figure 5.  
<img src="./graphics/ghost_image.png"/>


## Ethereum internals

Now let’s explore the main components that comprise the Ethereum blockchain.
 - Accounts
 - State
 - Gas and Fees
 - Transactions
 - Blocks
 - Transaction execution
 - Mining
 - Proof of work

## Accounts

The global “shared-state” of Ethereum is comprised of many small objects (“accounts”) that are able to interact with one another through a message-passing framework. Each account has a state associated with it and a 20-byte address. An address in Ethereum is a 160-bit identifier that is used to identify any account.

There are two types of accounts:
 - Externally owned accounts, which are controlled by private keys and have no code associated with them.
 - Contract accounts, which are controlled by their contract code and have code associated with them.
 
 ### Figure 6. ###
 
 <img src = "./graphics/accounts.png"/>

### Externally owned accounts vs. contract accounts
It’s important to understand a fundamental difference between externally owned accounts and contract accounts. An externally owned account can send messages to other externally owned accounts OR to other contract accounts by creating and signing a transaction using its private key. A message between two externally owned accounts is simply a value transfer. But a message from an externally owned account to a contract account activates the contract account’s code, allowing it to perform various actions (e.g. transfer tokens, write to internal storage, mint new tokens, perform some calculation, create new contracts, etc.).

**Note.** Unlike externally owned accounts, contract accounts can’t initiate new transactions on their own. Instead, contract accounts can only fire transactions in response to other transactions they have received (from an externally owned account or from another contract account). We’ll learn more about contract-to-contract calls in the “Transactions and Messages” section.

### Figure 7. ###
<img src="./graphics/accounts2.png"/>




Any action that occurs on the Ethereum blockchain is always set in motion by transactions fired from externally controlled accounts.

### Figure 8. ###

<img src="./graphics/account_blockchain.png"/>

## Account state
The account state consists of four components, which are present regardless of the type of account:

 - nonce: If the account is an externally owned account, this number represents the number of transactions sent from the account’s address. If the account is a contract account, the nonce is the number of contracts created by the account.
 - balance: The number of Wei owned by this address. There are 1e+18 Wei per Ether.
 - storageRoot: A hash of the root node of a Merkle Patricia tree. This tree encodes the hash of the storage contents of this account, and is empty by default.
 - codeHash: The hash of the EVM (Ethereum Virtual Machine — more on this later) code of this account. For contract accounts, this is the code that gets hashed and stored as the codeHash. For externally owned accounts, the codeHash field is the hash of the empty string.
 
 ### Figure 9. ###
 <img src="./graphics/account_state.png"/>

Ethereum’s global state consists of a mapping between account addresses and the account states. This mapping is stored in a data structure known as a *Merkle Patricia* tree.

A Merkle tree (or also referred as “Merkle trie”) is a type of binary tree composed of a set of nodes with:
 - a large number of leaf nodes at the bottom of the tree that contain the underlying data
 - a set of intermediate nodes, where each node is the hash of its two child nodes
 - a single root node, also formed from the hash of its two child node, representing the top of the tree

### Figure 10. ###

<img src = "./graphics/Merkle_Patricia_1.png"/>

The data at the bottom of the tree is generated by splitting the data that we want to store into *chunks*, then splitting the chunks into *buckets*, and then taking the hash of each bucket and repeating the same process until the total number of hashes remaining becomes only one: **the root hash**.

This tree is required to have a key for every value stored inside it. Beginning from the root node of the tree, the key should tell you which child node to follow to get to the corresponding value, which is stored in the leaf nodes. In Ethereum’s case, the key/value mapping for the state tree is between addresses and their associated accounts, including the balance, nonce, codeHash, and storageRoot for each account (where the storageRoot is itself a tree).

<img src="./graphics/ethereum_tree_.png"/>

This same trie structure is used also to store transactions and receipts. More specifically, every block has a “header” which stores the hash of the root node of three different Merkle trie structures, including:
 - State trie
 - Transactions trie
 - Receipts trie

<img src="./graphics/block_header.png"/>

The ability to store all this information efficiently in Merkle trees is incredibly useful in Ethereum for what we call “light clients” or “light nodes.” Remember that a blockchain is maintained by a bunch of nodes. Broadly speaking, there are two types of nodes: full nodes and light nodes.

A full archive node synchronizes the blockchain by downloading the full chain, from the genesis block to the current head block, executing all of the transactions contained within. Typically, miners store the full archive node, because they are required to do so for the mining process. It is also possible to download a full node without executing every transaction. Regardless, any full node contains the entire chain.

But unless a node needs to execute every transaction or easily query historical data, there’s really no need to store the entire chain. This is where the concept of a light node comes in. Instead of downloading and storing the full chain and executing all of the transactions, light nodes download only the chain of headers, from the genesis block to the current head, without executing any transactions or retrieving any associated state. Because light nodes have access to block headers, which contain hashes of three tries, they can still easily generate and receive verifiable answers about transactions, events, balances, etc.
The reason this works is because hashes in the Merkle tree propagate upward — if a malicious user attempts to swap a fake transaction into the bottom of a Merkle tree, this change will cause a change in the hash of the node above, which will change the hash of the node above that, and so on, until it eventually changes the root of the tree.


<img src="./graphics/merkle_tree_2.png"/>

Any node that wants to verify a piece of data can use something called a “Merkle proof” to do so. A Merkle proof consists of:
 - A chunk of data to be verified and its hash
 - The root hash of the tree
 - The “branch” (all of the partner hashes going up along the path from the chunk to the root)

Anyone reading the proof can verify that the hashing for that branch is consistent all the way up the tree, and therefore that the given chunk is actually at that position in the tree.

In summary, the benefit of using a Merkle Patricia tree is that the root node of this structure is cryptographically dependent on the data stored in the tree, and so the hash of the root node can be used as a secure identity for this data. Since the block header includes the root hash of the state, transactions, and receipts trees, any node can validate a small part of state of Ethereum without needing to store the entire state, which can be potentially unbounded in size.

## Gas and payment
One very important concept in Ethereum is the concept of fees. Every computation that occurs as a result of a transaction on the Ethereum network incurs a fee — there’s no free lunch! This fee is paid in a denomination called “gas.”
Gas is the unit used to measure the fees required for a particular computation. Gas price is the amount of Ether you are willing to spend on every unit of gas, and is measured in “gwei.” “Wei” is the smallest unit of Ether, where 1⁰¹⁸ Wei represents 1 Ether. One gwei is 1,000,000,000 Wei.

With every transaction, a sender sets a gas limit and gas price. The product of gas price and gas limit represents the maximum amount of Wei that the sender is willing to pay for executing a transaction.

For example, let’s say the sender sets the gas limit to 50,000 and a gas price to 20 gwei. This implies that the sender is willing to spend at most 50,000 x 20 gwei = 1,000,000,000,000,000 Wei = 0.001 Ether to execute that transaction.

<img src="./graphics/gas1.png"/>

Remember that the gas limit represents the maximum gas the sender is willing to spend money on. If they have enough Ether in their account balance to cover this maximum, they’re good to go. The sender is refunded for any unused gas at the end of the transaction, exchanged at the original rate.

<img src="./graphics/Ethereum_transaction1.png"/>